# A quick Echoshader demo

[Echoshader](https://github.com/OSOceanAcoustics/echoshader) is a package in the `Echostack` that provides interactive visualization functionalities.

In this notebook we demo the functionality to interactively investigate Sv distributions from an echogram.

For conveniece, we will reuse the MVBS dataset created from the [](quick_echopype_exercise) notebook.

:::{caution}
There are some dependency issues we still need to resolve for Echoshader, but we don't currently have resources to actively develop this package.
Therefore, if you want to try out this notebook and other Echoshader functionalities, please create a development environment following the instructions [here](https://github.com/OSOceanAcoustics/echoshader?tab=readme-ov-file#creating-a-development-environment). A simple `pip install` will likely fail. :(
:::

In [1]:
import panel
import xarray as xr
import numpy as np

import echoshader

In [2]:
fname = "./resources/Hake-D20230811-T165727_MVBS.zarr"
ds_MVBS = xr.open_dataset(fname, engine="zarr")

In [3]:
# Get the channel names
ds_MVBS["channel"].values

array(['WBT 400140-15 ES120-7C_ES', 'WBT 400141-15 ES18_ES',
       'WBT 400142-15 ES70-7C_ES', 'WBT 400143-15 ES38B_ES',
       'WBT 400145-15 ES200-7C_ES'], dtype='<U25')

In [4]:
# Echoshader likes channel and not frequency_nominal
ds_MVBS = ds_MVBS.swap_dims({"frequency_nominal": "channel"})

In [5]:
# Echoshader slider needs the actual_range attribute to work
ds_MVBS["Sv"].attrs["actual_range"] = [np.nanmax(ds_MVBS["Sv"].values), np.nanmin(ds_MVBS["Sv"].values)]

In [6]:
from holoviews import opts

In [7]:
# Only want to look at 3 channels
ds_MVBS = ds_MVBS.sel(channel = [
        "WBT 400141-15 ES18_ES",
        "WBT 400143-15 ES38B_ES",
        "WBT 400140-15 ES120-7C_ES",
    ]
)

In [8]:
# Plot interactive histogram
eg = ds_MVBS.eshader.echogram(
    channel = ["WBT 400143-15 ES38B_ES", "WBT 400140-15 ES120-7C_ES"],
    cmap = "viridis",
    vert_dim="depth",
    vmin=-80, vmax=-30,
    opts = opts.Image(width=500, height=300),
)

hist = ds_MVBS.eshader.hist(
    bins = 30, overlay = True,
)

table = ds_MVBS.eshader.table()

panel.Column(eg, hist, table)

Column
    [0] ParamMethod(method, _pane=HoloViews, defer_load=False)
    [1] ParamMethod(method, _pane=HoloViews, defer_load=False)
    [2] ParamMethod(method, _pane=HoloViews, defer_load=False)